In [1]:
from tensorflow.keras.applications.vgg16 import VGG16,preprocess_input,decode_predictions
from tensorflow.keras.preprocessing import image
import numpy as np
import cv2
import os
import random
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelBinarizer
from tensorflow.keras.utils import to_categorical
from imutils import paths
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import load_img
from tensorflow.keras.layers import Reshape

from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator,load_img
import matplotlib.pyplot as plt
from keras.models import Sequential, load_model
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
import PIL

In [2]:
vgg = VGG16()

In [3]:
vgg.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

As we know from above Vgg16 is functional model and to cut the model last layer and than

adding its whole layer to our last layer we need to convert it to Sequential model

Checking and cutting Vgg16 last layer by running for loop

In [4]:
for layer in vgg.layers[:-1]:
    print(layer.__class__.__name__)

InputLayer
Conv2D
Conv2D
MaxPooling2D
Conv2D
Conv2D
MaxPooling2D
Conv2D
Conv2D
Conv2D
MaxPooling2D
Conv2D
Conv2D
Conv2D
MaxPooling2D
Conv2D
Conv2D
Conv2D
MaxPooling2D
Flatten
Dense
Dense


now we can see last layer removed

now we removed last layer so we add this to our sequential layer for ladding our last layer

same as we added earlier in our sequential layer but this we add ready made layer with our last layer made by us

In [5]:
model = Sequential()

Converting functional layer to Sequential layer

In [6]:
for layer in vgg.layers[:-1]:
    model.add(layer)

In [7]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 56, 56, 256)       2

Now we can see last layer removed and model converted to Sequential

now we will freeze our layers because we don't want to change the weight durning 

backpropagation of our reday made layer(vgg16) we added

In [8]:
# freezing all layers
for layer in model.layers:
    layer.trainable = False

In [9]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 56, 56, 256)       2

As we can now see model is freeze and there is no trainable parameters in model summary

Adding last dense layer with only one node

In [10]:
model.add(Dense(2))
model.add(Activation('softmax'))

In [11]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 56, 56, 256)       2

In [12]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [13]:
training_path = r'C:\Users\sonuc\Desktop\Data_Science\Malaria_detection\Malaria Cells\training_set'
testing_path = r'C:\Users\sonuc\Desktop\Data_Science\Malaria_detection\Malaria Cells\testing_set'

In [16]:
batch_size = 10 

# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(rescale=1.0/255,
                                   rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   fill_mode='nearest')

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1./255)

# this is a generator that will read pictures found in
# subfolers of 'data/train', and indefinitely generate
# batches of augmented image data 
train_generator = train_datagen.flow_from_directory(
        training_path,  # this is the target directory
        target_size=(224, 224),  # all images will be resized to 150x150
        batch_size=batch_size)
        #class_mode='binary')  # since we use binary_crossentropy loss, we need binary labels

# this is a similar generator, for validation data
validation_generator = test_datagen.flow_from_directory(
        testing_path,
        target_size=(224, 224),
        batch_size=batch_size)#,
        #class_mode='binary')

Found 27558 images belonging to 2 classes.
Found 15832 images belonging to 2 classes.


In [17]:
model.fit(
        train_generator,
        steps_per_epoch=2000 // batch_size,
        epochs=50,
        validation_data=validation_generator,
        validation_steps=800 // batch_size)

model.save('malaria_detect_vvg16.h5')  # always save your weights after training or during training

Epoch 1/50
200/200 [==============================] - 32s 144ms/step - loss: 0.6737 - accuracy: 0.6005 - val_loss: 0.5516 - val_accuracy: 0.7837
Epoch 2/50
200/200 [==============================] - 32s 160ms/step - loss: 0.5619 - accuracy: 0.7097 - val_loss: 0.6898 - val_accuracy: 0.5562
Epoch 3/50
200/200 [==============================] - 29s 145ms/step - loss: 0.5315 - accuracy: 0.7370 - val_loss: 0.5495 - val_accuracy: 0.6675
Epoch 4/50
200/200 [==============================] - 29s 146ms/step - loss: 0.4729 - accuracy: 0.8015 - val_loss: 0.4210 - val_accuracy: 0.8400
Epoch 5/50
200/200 [==============================] - 29s 146ms/step - loss: 0.4838 - accuracy: 0.7670 - val_loss: 0.5187 - val_accuracy: 0.7150
Epoch 6/50
200/200 [==============================] - 29s 146ms/step - loss: 0.4655 - accuracy: 0.7810 - val_loss: 0.4305 - val_accuracy: 0.7912
Epoch 7/50
200/200 [==============================] - 29s 146ms/step - loss: 0.4410 - accuracy: 0.8110 - val_loss: 0.5435 - val_ac

In [20]:
model = load_model('malaria_detect_vvg16.h5') 

In [28]:
image_path = r'C:\Users\sonuc\Desktop\Data_Science\Malaria_detection\Malaria Cells\single_prediction\Uninfected.png'
image_path

'C:\\Users\\sonuc\\Desktop\\Data_Science\\Malaria_detection\\Malaria Cells\\single_prediction\\Uninfected.png'

In [29]:
user_img = image.load_img(image_path,target_size = (224,224))
user_img = image.img_to_array(user_img)
user_img = user_img.reshape(1, 224, 224, 3)


x = (model.predict(user_img) > 0.5).astype("int32")
#print(x)
if(x[0][0]== 0):      
    label = 'Uninfected'
    print(label)
else:
    label = 'Parasitised'
    print(label)

Uninfected
